<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Ni-Mo-data" data-toc-modified-id="Load-Ni-Mo-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Ni-Mo data</a></span></li><li><span><a href="#Set-up-the-MTP-and-train" data-toc-modified-id="Set-up-the-MTP-and-train-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Set up the MTP and train</a></span></li><li><span><a href="#Predict-and-evaluate-the-energies-and-forces-of-training-data" data-toc-modified-id="Predict-and-evaluate-the-energies-and-forces-of-training-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Predict and evaluate the energies and forces of training data</a></span></li><li><span><a href="#Write-and-load-fitted-mtp-with-parameters-files" data-toc-modified-id="Write-and-load-fitted-mtp-with-parameters-files-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Write and load fitted mtp with parameters files</a></span></li><li><span><a href="#Lattice-constants-and-eslastic-constants" data-toc-modified-id="Lattice-constants-and-eslastic-constants-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Lattice constants and eslastic constants</a></span></li><li><span><a href="#Surface-energy-calculation---Ni-as-an-example" data-toc-modified-id="Surface-energy-calculation---Ni-as-an-example-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Surface energy calculation - Ni as an example</a></span></li><li><span><a href="#Energy,-force,-stress-prediction" data-toc-modified-id="Energy,-force,-stress-prediction-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Energy, force, stress prediction</a></span></li></ul></div>

# Load Ni-Mo data

In [1]:
from pymatgen.core import Structure
from monty.serialization import loadfn

data = loadfn('data.json')
train_structures = [d['structure'] for d in data]
train_energies = [d['outputs']['energy'] for d in data]
train_forces = [d['outputs']['forces'] for d in data]
train_stresses = [d['outputs']['stress'] for d in data]

# Set up the MTP and train

In [2]:
from maml.apps.pes import MTPotential

mtp = MTPotential()

In [3]:
mtp.train(train_structures=train_structures, train_energies=train_energies,
          train_forces=train_forces, train_stresses = None, max_dist=5, stress_weight=0)

INFO:maml.utils._lammps:Structure index 0 is rotated.
INFO:maml.utils._lammps:Structure index 1 is rotated.
INFO:maml.utils._lammps:Structure index 2 is rotated.
INFO:maml.utils._lammps:Structure index 3 is rotated.
INFO:maml.utils._lammps:Structure index 4 is rotated.
INFO:maml.utils._lammps:Structure index 5 is rotated.
INFO:maml.utils._lammps:Structure index 6 is rotated.
INFO:maml.utils._lammps:Structure index 7 is rotated.
INFO:maml.utils._lammps:Structure index 8 is rotated.
INFO:maml.utils._lammps:Structure index 9 is rotated.


0

# Predict and evaluate the energies and forces of training data

In [4]:
df_orig, df_predict = mtp.evaluate(test_structures=train_structures, 
                                   test_energies=train_energies,
                                   test_forces=train_forces, test_stresses=train_stresses)

INFO:maml.utils._lammps:Structure index 0 is rotated.
INFO:maml.utils._lammps:Structure index 1 is rotated.
INFO:maml.utils._lammps:Structure index 2 is rotated.
INFO:maml.utils._lammps:Structure index 3 is rotated.
INFO:maml.utils._lammps:Structure index 4 is rotated.
INFO:maml.utils._lammps:Structure index 5 is rotated.
INFO:maml.utils._lammps:Structure index 6 is rotated.
INFO:maml.utils._lammps:Structure index 7 is rotated.
INFO:maml.utils._lammps:Structure index 8 is rotated.
INFO:maml.utils._lammps:Structure index 9 is rotated.


In [5]:
from sklearn.metrics import mean_absolute_error
import numpy as np

E_p = np.array(df_predict[df_predict['dtype'] == 'energy']['y_orig'])/df_predict[df_predict['dtype'] == 'energy']['n']
E_o = np.array(df_orig[df_orig['dtype'] == 'energy']['y_orig'])/df_orig[df_orig['dtype'] == 'energy']['n']
print("MAE of training energy prediction is {} meV/atom".format(mean_absolute_error(E_o,E_p)*1000))

F_p = np.array(df_predict[df_predict['dtype'] == 'force']['y_orig'])/df_predict[df_predict['dtype'] == 'force']['n']
F_o = np.array(df_orig[df_orig['dtype'] == 'force']['y_orig'])/df_orig[df_orig['dtype'] == 'force']['n']
print("MAE of training force prediction is {} eV/Å".format(mean_absolute_error(F_o,F_p)))

MAE of training energy prediction is 2.452442759843798 meV/atom
MAE of training force prediction is 0.12078751064814815 eV/Å


# Write and load fitted mtp with parameters files

In [6]:
mtp.write_param(fitted_mtp='fitted.mtp')

['pair_style        mlip mlip.ini', 'pair_coeff        * *']

In [7]:
mtp_loaded = MTPotential.from_config(filename='fitted.mtp', elements=["Ni", "Mo"])

# Lattice constants and eslastic constants
Large error due to limited training data -- 10 structures

In [8]:
from pymatgen.core import Lattice

Ni = Structure.from_spacegroup(sg='Fm-3m', species=['Ni'], lattice=Lattice.cubic(3.51), coords=[[0, 0, 0]])
Mo = Structure.from_spacegroup(sg='Im-3m', species=['Mo'], lattice=Lattice.cubic(3.17), coords=[[0, 0, 0]])

In [9]:
from maml.apps.pes import LatticeConstant

lc_calculator = LatticeConstant(ff_settings=mtp_loaded)
a, b, c = lc_calculator.calculate([Ni])[0]
print('Ni', 'Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

lc_calculator = LatticeConstant(ff_settings=mtp_loaded)
a, b, c = lc_calculator.calculate([Mo])[0]
print('Mo', 'Lattice a: {}, Lattice b: {}, Lattice c: {}'.format(a, b, c))

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_mpich
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_mpich
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_mpich
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_mpich


Ni Lattice a: 7.069140000000139, Lattice b: 7.069140000000139, Lattice c: 7.069140000000139
Mo Lattice a: 2.6113770640889977, Lattice b: 2.6113770640889977, Lattice c: 2.6113770640889977


In [10]:
from maml.apps.pes import ElasticConstant

Ni_ec_calculator = ElasticConstant(ff_settings=mtp_loaded)
Ni_C11, Ni_C12, Ni_C44, _ = Ni_ec_calculator.calculate([Ni])[0]
print('Ni', ' C11: ', Ni_C11, 'C12: ', Ni_C12, 'C44: ', Ni_C44)

Mo_ec_calculator = ElasticConstant(ff_settings=mtp_loaded)
Mo_C11, Mo_C12, Mo_C44, _ = Mo_ec_calculator.calculate([Mo])[0]
print('Mo', ' C11: ', Mo_C11, 'C12: ', Mo_C12, 'C44: ', Mo_C44)

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_mpich
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_mpich


Ni  C11:  20.7708419999108 C12:  25.8377555059269 C44:  71.3047001960825
Mo  C11:  -802479.953151941 C12:  -809354.693976045 C44:  -47578.8186817001


# Surface energy calculation - Ni as an example

In [11]:
from maml.apps.pes import SurfaceEnergy

mtp_loaded_Ni = MTPotential.from_config(filename='fitted.mtp.Ni', elements=["Ni"])
surface_e_calculator = SurfaceEnergy(ff_settings=mtp_loaded_Ni, 
                                     bulk_structure=Ni, 
                                     miller_indexes=[[1,0,0],[0,1,0],[1,1,0]])
results_surface = surface_e_calculator.calculate()
relaxed_surface_structures = [result[1] for result in results_surface]

print("Surface energys in Ni:")
for result in results_surface:
    print(f"Miller index: {result[0]}, surface energy: {result[2]} J/m^2, slab model has {result[1].num_sites} atoms")

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_mpich
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_mpich
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_mpich
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_mpich
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_mpich
INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_mpich


Surface energys in Ni:
Miller index: [1, 0, 0], surface energy: 2.231324198900004 J/m^2, slab model has 10 atoms
Miller index: [0, 1, 0], surface energy: 2.231324198900004 J/m^2, slab model has 10 atoms
Miller index: [1, 1, 0], surface energy: 2.3225616999482033 J/m^2, slab model has 14 atoms


# Energy, force, stress prediction

In [12]:
from maml.apps.pes import EnergyForceStress

efs_calculator = EnergyForceStress(ff_settings=mtp_loaded)
energy, forces, stresses = efs_calculator.calculate([train_structures[0]])[0]

print('energy: {}'.format(energy))
print('forces: \n', forces)
print('stresses: ', stresses)

INFO:maml.apps.pes._lammps:Setting Lammps executable to lmp_g++_mpich


energy: -1022.31654318373
forces: 
 [[-1.72880186e-01  5.63705118e-01 -1.57002643e-02]
 [ 5.74738859e-01 -3.21596709e-01 -7.49036066e-01]
 [-4.02881585e-01 -7.04802447e-02  5.89548087e-01]
 [ 1.99652041e-01  4.52133229e-01 -4.24866477e-01]
 [-6.10037223e-01  4.86436325e-02 -1.51852819e+00]
 [ 5.13114052e-01  6.84385142e-02 -4.44337513e-01]
 [-7.43415190e-01  2.18115425e-01  3.31340952e-01]
 [-7.67966228e-02  8.94474793e-02  7.21359382e-01]
 [-1.30463005e+00 -4.81045782e-01  1.93630096e-01]
 [ 8.08479492e-01 -2.94776926e-01 -5.49186032e-01]
 [ 1.45982577e-01 -4.56985945e-02 -6.89455782e-02]
 [-7.88895413e-01  1.38293460e-01 -5.12515089e-01]
 [-4.29616235e-02  3.60306921e-02  5.96081337e-01]
 [-1.59538050e-01 -5.58814957e-01 -5.68617332e-02]
 [ 2.59194144e-01 -7.15408807e-01 -1.35209746e+00]
 [-5.43740988e-01 -5.12669729e-01  3.83055184e-02]
 [-1.78625146e-01  1.14795712e-02 -1.51407451e-01]
 [-2.67453426e-01  5.93172998e-02 -6.08630657e-01]
 [-8.73032740e-03 -3.72299904e-01  1.24961306e